In [50]:
import pandas as pd

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace(',', '.', regex=True, inplace=True)

In [51]:
managers = prolongations["AM"].unique()

In [52]:
all_months = ['ноябрь 2022', 'декабрь 2022', 'январь 2023', 'февраль 2023',
              'март 2023', 'апрель 2023', 'май 2023', 'июнь 2023', 'июль 2023',
              'август 2023', 'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023',
              'декабрь 2023', 'январь 2024', 'февраль 2024']

months_to_calc_prolongation = ['январь 2023', 'февраль 2023', 'март 2023', 'апрель 2023',
                  'май 2023', 'июнь 2023', 'июль 2023', 'август 2023',
                  'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023', 'декабрь 2023']

In [53]:
# для начала посчитаем коэффициенты пролонгации для каждого менеджера:
# 1) за каждый месяц (2023 год)
# 2) за год (2023 год)

In [54]:
def capitalize_first_letter(string):
    return string[0].upper() + string[1:]

def replace_zeros_with_previous(df):
    for index, row in df.iterrows():
        previous_value = None
        for column in df.columns[1:]:
            if row[column] != 0:
                previous_value = row[column]
            elif row[column] == 0 and previous_value is not None:
                df.at[index, column] = previous_value
    
    return df

def remove_rows_with_zeros(df):
    return df[~df.apply(lambda row: (row == 0).any(), axis=1)]

In [55]:
managers_prolongations = {}

In [56]:
import numpy as np

for manager in managers:
    k1_up_sum_year = 0
    k1_down_sum_year = 0
    k2_up_sum_year = 0
    k2_down_sum_year = 0
    
    for prolongation_month in months_to_calc_prolongation:
        prolongation_month_index = all_months.index(prolongation_month)
        month0_index = all_months.index(prolongation_month) - 2
        month1_index = all_months.index(prolongation_month) - 1
        month0 = all_months[month0_index]
        month1 = all_months[month1_index]

        manager_completed_projects = prolongations[prolongations['AM'] == manager]
        
        month0_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month0]
        month1_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month1]

        
        # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 0 (КОЭФФИЦИЕНТ K2)
        
        k2_up_sum = 0
        k2_down_sum = 0
        
        ids = month0_completed_projects['id'].unique()
        projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()

        # собираем индексы строк, которые нужно удалить
        to_drop = []

        for _, project in projects_financial_data.iterrows():
            for check_month in range(0, prolongation_month_index + 1):
                status = project[capitalize_first_letter(all_months[check_month])]
                if status in ["стоп", "end"]:
                    to_drop.append(project.name)
                    break

        projects_financial_data.drop(index=to_drop, inplace=True)
        
        required_months = all_months[0: prolongation_month_index + 1]
        required_months = [capitalize_first_letter(month) for month in required_months]
        required_months = ['id'] + required_months

        projects_financial_data = projects_financial_data[required_months]

        projects_financial_data = projects_financial_data.replace("в ноль", 0)
        projects_financial_data = projects_financial_data.replace("NaN", np.nan)
        projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')

        projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
        projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
        projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)
        
        print("Итоговый датафрейм для менеджера", manager, "за prolongation месяц", prolongation_month)
        print(projects_financial_data_grouped)
        
        # k2 up = сумма отгрузки всех проектов пролонгированных во второй месяц после завершения, например сумма отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле, но есть в мае (за май).
        # k2 down = сумма отгрузки последнего месяца проектов не пролонгированных в первый, например сумма отгрузки проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март)
        # нет отгрузки == NaN
        for index, row in projects_financial_data_grouped.iterrows():
            if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(prolongation_month)] > 0 and row[capitalize_first_letter(month0)] > 0:
                print(f"Прибавляем {row[capitalize_first_letter(prolongation_month)]} к k2_up_sum")
                k2_up_sum += row[capitalize_first_letter(prolongation_month)]
            if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(month0)] > 0:
                print(f"Прибавляем {row[capitalize_first_letter(month0)]} к k2_down_sum")
                k2_down_sum += row[capitalize_first_letter(month0)]
        
        k2_up_sum_year += k2_up_sum
        k2_down_sum_year += k2_down_sum
        
        if k2_down_sum > 0:
            k2 = k2_up_sum / k2_down_sum
        else:
            k2 = 0
            
        if manager not in managers_prolongations:
            managers_prolongations[manager] = {}
        if prolongation_month not in managers_prolongations[manager]:
            managers_prolongations[manager][prolongation_month] = {}
        managers_prolongations[manager][prolongation_month]['k2'] = k2
        
        
        # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 1 (КОЭФФИЦИЕНТ K1)
        
        k1_up_sum = 0
        k1_down_sum = 0
        
        ids = month1_completed_projects['id'].unique()
        projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()

        # собираем индексы строк, которые нужно удалить
        to_drop = []
        
        for _, project in projects_financial_data.iterrows():
            for check_month in range(0, prolongation_month_index + 1):
                status = project[capitalize_first_letter(all_months[check_month])]
                if status in ["стоп", "end"]:
                    to_drop.append(project.name)
                    break
        
        projects_financial_data.drop(index=to_drop, inplace=True)
                    
        required_months = all_months[0: prolongation_month_index + 1]
        required_months = [capitalize_first_letter(month) for month in required_months]
        required_months = ['id'] + required_months
        
        projects_financial_data = projects_financial_data[required_months]
        
        projects_financial_data = projects_financial_data.replace("в ноль", 0)
        projects_financial_data = projects_financial_data.replace("NaN", np.nan)
        projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')
        
        projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
        projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
        projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)
        
        print("Итоговый датафрейм для менеджера", manager, "за prolongation месяц", prolongation_month)
        print(projects_financial_data_grouped)
        
        # k1 up = сумма отгрузки всех проектов пролонгированных в первый месяц после завершения, например сумма отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май)
        # k1 down = сумма отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов, например сумма отгрузки проектов, завершившихся в апреле (за апрель)
        # нет отгрузки == NaN
        for index, row in projects_financial_data_grouped.iterrows():
            if row[capitalize_first_letter(month1)] > 0 and row[capitalize_first_letter(prolongation_month)] > 0:
                print(f"Прибавляем {row[capitalize_first_letter(prolongation_month)]} к k1_up_sum")
                k1_up_sum += row[capitalize_first_letter(prolongation_month)]
            if row[capitalize_first_letter(month1)] > 0:
                print(f"Прибавляем {row[capitalize_first_letter(month1)]} к k1_down_sum")
                k1_down_sum += row[capitalize_first_letter(month1)]
                
        k1_up_sum_year += k1_up_sum
        k1_down_sum_year += k1_down_sum
                
        if k1_down_sum > 0:
            k1 = k1_up_sum / k1_down_sum
        else:
            k1 = 0
            
        managers_prolongations[manager][prolongation_month]['k1'] = k1

    if k2_down_sum_year > 0:
        k2 = k2_up_sum_year / k2_down_sum_year
    else:
        k2 = 0

    if k1_down_sum_year > 0:
        k1 = k1_up_sum_year / k1_down_sum_year
    else:
        k1 = 0

    managers_prolongations[manager]["year"] = {}
    managers_prolongations[manager]["year"]["k1"] = k1
    managers_prolongations[manager]["year"]["k2"] = k2

Итоговый датафрейм для менеджера Васильев Артем Александрович за prolongation месяц январь 2023
     Ноябрь 2022  Декабрь 2022  Январь 2023
id                                         
42       36220.0           NaN          NaN
369      42600.0        7450.0          NaN
592     104642.0           NaN          NaN
600     281417.0      175100.0     267220.0
633      40200.0       40200.0      40200.0
665      10000.0           NaN          NaN
666     110000.0           NaN          NaN
Прибавляем 36220.0 к k2_down_sum
Прибавляем 104642.0 к k2_down_sum
Прибавляем 10000.0 к k2_down_sum
Прибавляем 110000.0 к k2_down_sum
Итоговый датафрейм для менеджера Васильев Артем Александрович за prolongation месяц январь 2023
     Ноябрь 2022  Декабрь 2022  Январь 2023
id                                         
31      55100.00      55100.00          NaN
46      37939.50      37939.50      39838.5
49      85842.00      85842.00      83358.0
228     28200.00      28200.00      33885.0
471     89185.

In [57]:
print(managers_prolongations)

{'Васильев Артем Александрович': {'январь 2023': {'k2': 0.0, 'k1': 0.5981708037069663}, 'февраль 2023': {'k2': 0.05458136005412423, 'k1': 1.056562603886488}, 'март 2023': {'k2': 0.7078232898172324, 'k1': 0.7479734940553308}, 'апрель 2023': {'k2': 0.21480670068663335, 'k1': 0.4432843067573071}, 'май 2023': {'k2': 0.0, 'k1': 0.39939681269089256}, 'июнь 2023': {'k2': 0.061481622354117915, 'k1': 0.38928703566102807}, 'июль 2023': {'k2': 0.0, 'k1': 0.5850156702127803}, 'август 2023': {'k2': 0.0, 'k1': 0.47750385208012325}, 'сентябрь 2023': {'k2': 0.0, 'k1': 0.172432592799243}, 'октябрь 2023': {'k2': 0.07225635943178065, 'k1': 0.8867503799486758}, 'ноябрь 2023': {'k2': 0.0, 'k1': 0.6210006947226171}, 'декабрь 2023': {'k2': 0.48485855522784865, 'k1': 0.32406316285646947}, 'year': {'k1': 0.5291744774064999, 'k2': 0.07042621684151683}}, 'Михайлов Андрей Сергеевич': {'январь 2023': {'k2': 0.0, 'k1': 0.6879234191303456}, 'февраль 2023': {'k2': 0.0, 'k1': 0.9036449656826814}, 'март 2023': {'k2': 0

In [58]:
# мы посчитали коэффициенты пролонгации для каждого менеджера за каждый месяц и за год
# для всего отдела в целом также посчитаем коэффициенты пролонгации за каждый месяц и за год:

department_prolongations = {}

k1_up_sum_year = 0
k1_down_sum_year = 0
k2_up_sum_year = 0
k2_down_sum_year = 0

for prolongation_month in months_to_calc_prolongation:
    prolongation_month_index = all_months.index(prolongation_month)
    month0_index = all_months.index(prolongation_month) - 2
    month1_index = all_months.index(prolongation_month) - 1
    month0 = all_months[month0_index]
    month1 = all_months[month1_index]

    department_completed_projects = prolongations
    
    month0_completed_projects = department_completed_projects[department_completed_projects['month'] == month0]
    month1_completed_projects = department_completed_projects[department_completed_projects['month'] == month1]
    
    
    # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 0 (КОЭФФИЦИЕНТ K2)
    
    k2_up_sum = 0
    k2_down_sum = 0
    
    ids = month0_completed_projects['id'].unique()
    projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
    
    # собираем индексы строк, которые нужно удалить
    to_drop = []
    
    for _, project in projects_financial_data.iterrows():
        for check_month in range(0, prolongation_month_index + 1):
            status = project[capitalize_first_letter(all_months[check_month])]
            if status in ["стоп", "end"]:
                to_drop.append(project.name)
                break
                
    projects_financial_data.drop(index=to_drop, inplace=True)
    
    required_months = all_months[0: prolongation_month_index + 1]
    required_months = [capitalize_first_letter(month) for month in required_months]
    required_months = ['id'] + required_months
    
    projects_financial_data = projects_financial_data[required_months]
    
    projects_financial_data = projects_financial_data.replace("в ноль", 0)
    projects_financial_data = projects_financial_data.replace("NaN", np.nan)
    projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')
    
    projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
    projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
    projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)
    
    print("Итоговый датафрейм для отдела за prolongation месяц", prolongation_month)
    print(projects_financial_data_grouped)
    
    # k2 up = сумма отгрузки всех проектов пролонгированных во второй месяц после завершения, например сумма отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле, но есть в мае (за май).
    # k2 down = сумма отгрузки последнего месяца проектов не пролонгированных в первый, например сумма отгрузки проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март)
    # нет отгрузки == NaN
    for index, row in projects_financial_data_grouped.iterrows():
        if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(prolongation_month)] > 0 and row[capitalize_first_letter(month0)] > 0:
            print(f"Прибавляем {row[capitalize_first_letter(prolongation_month)]} к k2_up_sum")
            k2_up_sum += row[capitalize_first_letter(prolongation_month)]
        if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(month0)] > 0:
            print(f"Прибавляем {row[capitalize_first_letter(month0)]} к k2_down_sum")
            k2_down_sum += row[capitalize_first_letter(month0)]
            
    k2_up_sum_year += k2_up_sum
    k2_down_sum_year += k2_down_sum
    
    if k2_down_sum > 0:
        k2 = k2_up_sum / k2_down_sum
    else:
        k2 = 0
    
    if "year" not in department_prolongations:
        department_prolongations["year"] = {}
    if prolongation_month not in department_prolongations:
        department_prolongations[prolongation_month] = {}
    department_prolongations[prolongation_month]['k2'] = k2
    
    
    # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 1 (КОЭФФИЦИЕНТ K1)
    
    k1_up_sum = 0
    k1_down_sum = 0
    
    ids = month1_completed_projects['id'].unique()
    projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
    
    # собираем индексы строк, которые нужно удалить
    to_drop = []
    
    for _, project in projects_financial_data.iterrows():
        for check_month in range(0, prolongation_month_index + 1):
            status = project[capitalize_first_letter(all_months[check_month])]
            if status in ["стоп", "end"]:
                to_drop.append(project.name)
                break
    
    projects_financial_data.drop(index=to_drop, inplace=True)
    
    required_months = all_months[0: prolongation_month_index + 1]
    required_months = [capitalize_first_letter(month) for month in required_months]
    required_months = ['id'] + required_months
    
    projects_financial_data = projects_financial_data[required_months]
    
    projects_financial_data = projects_financial_data.replace("в ноль", 0)
    projects_financial_data = projects_financial_data.replace("NaN", np.nan)
    projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')
    
    projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
    projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
    projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)
    
    print("Итоговый датафрейм для отдела за prolongation месяц", prolongation_month)
    print(projects_financial_data_grouped)
    
    # k1 up = сумма отгрузки всех проектов пролонгированных в первый месяц после завершения, например сумма отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май)
    # k1 down = сумма отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов, например сумма отгрузки проектов, завершившихся в апреле (за апрель)
    # нет отгрузки == NaN
    for index, row in projects_financial_data_grouped.iterrows():
        if row[capitalize_first_letter(month1)] > 0 and row[capitalize_first_letter(prolongation_month)] > 0:
            print(f"Прибавляем {row[capitalize_first_letter(prolongation_month)]} к k1_up_sum")
            k1_up_sum += row[capitalize_first_letter(prolongation_month)]
        if row[capitalize_first_letter(month1)] > 0:
            print(f"Прибавляем {row[capitalize_first_letter(month1)]} к k1_down_sum")
            k1_down_sum += row[capitalize_first_letter(month1)]
            
    k1_up_sum_year += k1_up_sum
    k1_down_sum_year += k1_down_sum
    
    if k1_down_sum > 0:
        k1 = k1_up_sum / k1_down_sum
    else:
        k1 = 0
        
    department_prolongations[prolongation_month]['k1'] = k1
    
if k2_down_sum_year > 0:
    k2 = k2_up_sum_year / k2_down_sum_year
else:
    k2 = 0
    
if k1_down_sum_year > 0:
    k1 = k1_up_sum_year / k1_down_sum_year
else:
    k1 = 0
    
department_prolongations["year"]["k1"] = k1
department_prolongations["year"]["k2"] = k2

Итоговый датафрейм для отдела за prolongation месяц январь 2023
     Ноябрь 2022  Декабрь 2022  Январь 2023
id                                         
42       36220.0           NaN          NaN
87       70050.0           NaN      73380.0
301      55220.0       40830.0      40830.0
369      42600.0        7450.0          NaN
419          NaN           NaN          NaN
429      30280.0       35580.0      35830.0
547     144180.0      104660.0     226420.0
548     674000.0      674000.0     674000.0
578      82800.0           NaN          NaN
586      52240.0           NaN          NaN
592     104642.0           NaN          NaN
600     281417.0      175100.0     267220.0
633      40200.0       40200.0      40200.0
637      38045.0           NaN          NaN
645      58060.0       46135.0          NaN
665      10000.0           NaN          NaN
666     110000.0           NaN          NaN
671      28000.0      105805.0     108230.0
704      41025.0           NaN          NaN
Прибавляем 3

In [59]:
print(department_prolongations)

{'year': {'k1': 0.5573933532314023, 'k2': 0.06756613440624357}, 'январь 2023': {'k2': 0.13463676695619625, 'k1': 0.45901597210351375}, 'февраль 2023': {'k2': 0.05005954574452784, 'k1': 0.7826135037295495}, 'март 2023': {'k2': 0.16166210920685742, 'k1': 0.7305092568287479}, 'апрель 2023': {'k2': 0.12872694956676295, 'k1': 0.5030577576404707}, 'май 2023': {'k2': 0.0, 'k1': 0.5526248138252068}, 'июнь 2023': {'k2': 0.03919222374339124, 'k1': 0.24989189619873936}, 'июль 2023': {'k2': 0.07948968686470759, 'k1': 0.5244888702316577}, 'август 2023': {'k2': 0.03470257886627872, 'k1': 0.48720696345385334}, 'сентябрь 2023': {'k2': 0.0, 'k1': 0.32862651739463433}, 'октябрь 2023': {'k2': 0.10918520716582525, 'k1': 0.817677564229945}, 'ноябрь 2023': {'k2': 0.018931474326249177, 'k1': 0.6394305427107582}, 'декабрь 2023': {'k2': 0.2612412612355543, 'k1': 0.5445954213860256}}


In [60]:
import xlsxwriter
import os

if not os.path.exists('results'):
    os.makedirs('results')
workbook = xlsxwriter.Workbook('results/prolongations.xlsx')

managers_sheet_months = workbook.add_worksheet('Managers_months')
managers_sheet_year = workbook.add_worksheet('Managers_year')
department_sheet = workbook.add_worksheet('Department')

managers_sheet_months.write(0, 0, "Менеджер")
managers_sheet_months.write(0, 1, "Месяц")
managers_sheet_months.write(0, 2, "K1")
managers_sheet_months.write(0, 3, "K2")

row = 1
for manager, prolongations in managers_prolongations.items():
    for month, values in prolongations.items():
        if month == "year":
            continue
        managers_sheet_months.write(row, 0, manager)
        managers_sheet_months.write(row, 1, month)
        managers_sheet_months.write(row, 2, values['k1'])
        managers_sheet_months.write(row, 3, values['k2'])
        row += 1
    
managers_sheet_year.write(0, 0, "Менеджер")
managers_sheet_year.write(0, 1, "Год")
managers_sheet_year.write(0, 2, "K1")
managers_sheet_year.write(0, 3, "K2")

row = 1
for manager, prolongations in managers_prolongations.items():
    managers_sheet_year.write(row, 0, manager)
    managers_sheet_year.write(row, 1, "2023 год")
    managers_sheet_year.write(row, 2, prolongations['year']['k1'])
    managers_sheet_year.write(row, 3, prolongations['year']['k2'])
    row += 1
    
department_sheet.write(0, 0, "Месяц")
department_sheet.write(0, 1, "K1")
department_sheet.write(0, 2, "K2")
row = 1
for month, values in department_prolongations.items():
    if month == "year":
        continue
    department_sheet.write(row, 0, month)
    department_sheet.write(row, 1, values['k1'])
    department_sheet.write(row, 2, values['k2'])
    row += 1

department_sheet.write(row, 0, "2023 год")
department_sheet.write(row, 1, department_prolongations['year']['k1'])
department_sheet.write(row, 2, department_prolongations['year']['k2'])
workbook.close()